In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

eye_model = load_model("C:\\Users\\admin\\Downloads\\model_trained_on_OACE.h5")
face_model=load_model("C:\\Users\\admin\\Downloads\\new_driver_drowsiness4.h5")

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade  = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

cap = cv2.VideoCapture(0)

def process_eye(eye_frame):
    resized = cv2.resize(eye_frame, (150, 150))
    gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
    gray = np.expand_dims(gray, axis=-1)
    normalized = gray.astype(np.float32) / 255.0
    input_tensor = np.expand_dims(normalized, axis=0)
    return input_tensor

def process_face(face):
    resized = cv2.resize(face, (224, 224))
    gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
    gray = np.expand_dims(gray, axis=-1)
    normalized = gray.astype(np.float32) / 255.0
    input_tensor = np.expand_dims(normalized, axis=0)
    return input_tensor


while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_frame, 1.3, 5)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)  # Blue box around face

        roi_gray = gray_frame[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]

        # ========== FACE-BASED MODEL (uses Haar-detected face) ==========
        face_img = roi_color.copy()
        if face_img.size != 0:
            input_face_tensor = process_face(face_img)
            face_pred = face_model.predict(input_face_tensor)[0][0]
            face_label = "Sleepy" if face_pred < 0.6 else "Not Sleepy"

            # Display on top-center of the screen
            text_position = (int(frame.shape[1] / 2) - 100, 40)
            cv2.putText(frame, f'Face status: {face_label} ({face_pred:.2f})',
                        text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.9,
                        (0, 255, 255) if face_label == "Not Sleepy" else (0, 0, 255), 2)

        # ========== EYE-BASED MODEL ==========
        eyes = eye_cascade.detectMultiScale(roi_gray)
        closed_eyes = 0
        total_eyes = 0

        for (ex, ey, ew, eh) in eyes[:2]:
            eye_img = roi_color[ey:ey+eh, ex:ex+ew]
            if eye_img.size == 0:
                continue

            input_tensor = process_eye(eye_img)
            pred = eye_model.predict(input_tensor)[0][0]
            label = "Open" if pred > 0.5 else "Closed"

            if label == "Closed":
                closed_eyes += 1
            total_eyes += 1

            cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)
            cv2.putText(roi_color, f'{label} ({pred:.2f})', (ex, ey - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)

        
        if  closed_eyes>=1:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(frame, "Sleepy", (30, 80),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        elif total_eyes!=0:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0,255, 0), 2)
            cv2.putText(frame, "Not Sleepy", (30, 80),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow("Sleep Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

model = load_model("C:\\Users\\admin\\Downloads\\new_driver_drowsiness.h5")

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)

def process(face_img):
    resized = cv2.resize(face_img, (224, 224))
    gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
    gray = np.expand_dims(gray, axis=-1)
    normalized = gray.astype(np.float32) / 255.0
    input_tensor = np.expand_dims(normalized, axis=0)  # (1, 256, 256, 1)
    return input_tensor

while True:
    ret, frame = cap.read()
    if not ret:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face_img = frame[y:y+h, x:x+w]

        if face_img.size == 0:
            continue

        input_tensor = process(face_img)
        prediction = model.predict(input_tensor)[0][0]
        label = "Not Sleepy" if prediction > 0.6 else "Sleepy"
        
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, f'{label} ({prediction:.2f})', (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)

    cv2.imshow("Live Face Detection + Prediction (Press 'y' to quit)", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [57]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load trained models
eye_model = load_model("C:\\Users\\admin\\Downloads\\model_trained_on_OACE.h5")
face_model = load_model("C:\\Users\\admin\\Downloads\\new_driver_drowsiness4.h5")

# Haar cascades
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade  = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Open webcam
cap = cv2.VideoCapture(0)

# Preprocessing functions
def process_eye(eye_frame):
    resized = cv2.resize(eye_frame, (150, 150))
    gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
    gray = np.expand_dims(gray, axis=-1)
    return np.expand_dims(gray.astype(np.float32) / 255.0, axis=0)

def process_face(face):
    resized = cv2.resize(face, (224, 224))
    gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
    gray = np.expand_dims(gray, axis=-1)
    return np.expand_dims(gray.astype(np.float32) / 255.0, axis=0)

# Alert state
drowsy_score = 0
alert_active = False
awaiting_acknowledgment = False

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_frame, 1.3, 5)

    drowsy_this_frame = False
    face_status_text = "Face: Unknown"
    face_status_color = (200, 200, 200)
    alert_status_text = "Alert: Not Sleepy"
    alert_status_color = (0, 255, 0)
    box_color = (0, 255, 0)

    for (x, y, w, h) in faces:
        roi_gray = gray_frame[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        face_img = roi_color.copy()

        # Face prediction
        face_pred = face_model.predict(process_face(face_img), verbose=0)[0][0]
        face_status_text = "Face: Sleepy" if face_pred < 0.6 else "Face: Not Sleepy"
        face_status_color = (0, 0, 255) if face_pred < 0.6 else (0, 255, 0)

        # Eye prediction
        eyes = eye_cascade.detectMultiScale(roi_gray)
        eyes = sorted(eyes, key=lambda e: e[0])[:2]
        closed_eyes = 0
        eyes_detected = False

        for (ex, ey, ew, eh) in eyes:
            eye_img = roi_color[ey:ey+eh, ex:ex+ew]
            if eye_img.size == 0:
                continue

            eyes_detected = True
            pred = eye_model.predict(process_eye(eye_img), verbose=0)[0][0]
            label = "Open" if pred > 0.5 else "Closed"
            if label == "Closed":
                closed_eyes += 1

            # Draw eye box using alert box color
            cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), box_color, 2)
            

        # Drowsiness decision
        if eyes_detected:
            if closed_eyes >= 1:
                drowsy_this_frame = True
        else:
            if face_pred < 0.6:
                drowsy_this_frame = True

        # Alert status update
        if drowsy_this_frame:
            alert_status_text = "Face: Sleepy"
            alert_status_color = (0, 0, 255)
            box_color = (0, 0, 255)
        else:
            alert_status_text = "Face: Not Sleepy"
            alert_status_color = (0, 255, 0)
            box_color = (0, 255, 0)

        # Score logic
        drowsy_score = drowsy_score + 1 if drowsy_this_frame else max(0, drowsy_score - 1)

        # Alert activation
        if drowsy_score > 8 and not awaiting_acknowledgment:
            alert_active = True
            awaiting_acknowledgment = True

        # Draw face box using alert status color
        cv2.rectangle(frame, (x, y), (x + w, y + h), box_color, 2)

        if alert_active:
            cv2.putText(frame, "!!! DROWSY ALERT !!! Press ENTER to confirm you're awake.",
                        (40, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    # Display statuses
    cv2.putText(frame, alert_status_text, (int(frame.shape[1] / 2) - 150, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, alert_status_color, 2)

    cv2.imshow("Sleep Detection", frame)

    key = cv2.waitKey(1)
    if key & 0xFF == ord('q'):
        break
    elif key == 13 and alert_active:  # ENTER
        alert_active = False
        awaiting_acknowledgment = False
        drowsy_score = 0
        print("Alert acknowledged. Monitoring resumed.")

cap.release()
cv2.destroyAllWindows()


Alert acknowledged. Monitoring resumed.
Alert acknowledged. Monitoring resumed.
Alert acknowledged. Monitoring resumed.
Alert acknowledged. Monitoring resumed.
Alert acknowledged. Monitoring resumed.
Alert acknowledged. Monitoring resumed.
Alert acknowledged. Monitoring resumed.
Alert acknowledged. Monitoring resumed.
Alert acknowledged. Monitoring resumed.
Alert acknowledged. Monitoring resumed.
